In [1]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, Latex
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np

In [3]:
class HiddenMarkov:
    def forward(self, Q, V, A, B, O, PI):  # 使用前向算法
        N = len(Q)  #可能存在的状态数量
        M = len(O)  # 观测序列的大小
        alphas = np.zeros((N, M))  # alpha值
        T = M  # 有几个时刻，有几个观测序列，就有几个时刻
        for t in range(T):  # 遍历每一时刻，算出alpha值
            indexOfO = V.index(O[t])  # 找出序列对应的索引
            for i in range(N):
                if t == 0:  # 计算初值
                    alphas[i][t] = PI[t][i] * B[i][indexOfO]  # P176（10.15）
                    print(
                        'alpha1(%d)=p%db%db(o1)=%f' % (i, i, i, alphas[i][t]))
                else:
                    alphas[i][t] = np.dot(
                        [alpha[t - 1] for alpha in alphas],
                        [a[i] for a in A]) * B[i][indexOfO]  # 对应P176（10.16）
                    print('alpha%d(%d)=[sigma alpha%d(i)ai%d]b%d(o%d)=%f' %
                          (t, i, t - 1, i, i, t, alphas[i][t]))
                    # print(alphas)
        P = np.sum([alpha[M - 1] for alpha in alphas])  # P176(10.17)
        # alpha11 = pi[0][0] * B[0][0]    #代表a1(1)
        # alpha12 = pi[0][1] * B[1][0]    #代表a1(2)
        # alpha13 = pi[0][2] * B[2][0]    #代表a1(3)

    def backward(self, Q, V, A, B, O, PI):  # 后向算法
        N = len(Q)  # 可能存在的状态数量
        M = len(O)  # 观测序列的大小
        betas = np.ones((N, M))  # beta
        for i in range(N):
            print('beta%d(%d)=1' % (M, i))
        for t in range(M - 2, -1, -1):
            indexOfO = V.index(O[t + 1])  # 找出序列对应的索引
            for i in range(N):
                betas[i][t] = np.dot(
                    np.multiply(A[i], [b[indexOfO] for b in B]),
                    [beta[t + 1] for beta in betas])
                realT = t + 1
                realI = i + 1
                print(
                    'beta%d(%d)=[sigma a%djbj(o%d)]beta%d(j)=(' %
                    (realT, realI, realI, realT + 1, realT + 1),
                    end='')
                for j in range(N):
                    print(
                        "%.2f*%.2f*%.2f+" % (A[i][j], B[j][indexOfO],
                                             betas[j][t + 1]),
                        end='')
                print("0)=%.3f" % betas[i][t])
        # print(betas)
        indexOfO = V.index(O[0])
        P = np.dot(
            np.multiply(PI, [b[indexOfO] for b in B]),
            [beta[0] for beta in betas])
        print("P(O|lambda)=", end="")
        for i in range(N):
            print(
                "%.1f*%.1f*%.5f+" % (PI[0][i], B[i][indexOfO], betas[i][0]),
                end="")
        print("0=%f" % P)

    def viterbi(self, Q, V, A, B, O, PI):
        OUTPUT = False
        N = len(Q)  #可能存在的状态数量
        M = len(O)  # 观测序列的大小
        deltas = np.zeros((N, M))
        psis = np.zeros((N, M))
        I = np.zeros((1, M))
        for t in range(M):
            realT = t + 1
            indexOfO = V.index(O[t])  # 找出序列对应的索引
            for i in range(N):
                realI = i + 1
                if t == 0:
                    deltas[i][t] = PI[0][i] * B[i][indexOfO]
                    psis[i][t] = 0
                    if OUTPUT:
                        print('delta1(%d)=pi%d * b%d(o1)=%.2f * %.2f=%.2f' %
                              (realI, realI, realI, PI[0][i], B[i][indexOfO],
                               deltas[i][t]))
                        print('psis1(%d)=0' % (realI))
                else:
                    deltas[i][t] = np.max(
                        np.multiply([delta[t - 1] for delta in deltas],
                                    [a[i] for a in A])) * B[i][indexOfO]
                    if OUTPUT:
                        print(
                            'delta%d(%d)=max[delta%d(j)aj%d]b%d(o%d)=%.2f*%.2f=%.5f'
                            % (realT, realI, realT - 1, realI, realI, realT,
                               np.max(
                                   np.multiply([delta[t - 1] for delta in deltas],
                                               [a[i] for a in A])), B[i][indexOfO],
                               deltas[i][t]))
                    psis[i][t] = np.argmax(
                        np.multiply(
                            [delta[t - 1] for delta in deltas],
                            [a[i]
                             for a in A])) + 1  #由于其返回的是索引，因此应+1才能和正常的下标值相符合。
                    if OUTPUT:
                        print('psis%d(%d)=argmax[delta%d(j)aj%d]=%d' %
                              (realT, realI, realT - 1, realI, psis[i][t]))
        if OUTPUT:
            print(deltas)
            print(psis)
        I[0][M - 1] = np.argmax([delta[M - 1] for delta in deltas]) + 1  
        #由于其返回的是索引，因此应+1才能和正常的下标值相符合。
        if OUTPUT:
            print('i%d=argmax[deltaT(i)]=%d' % (M, I[0][M - 1]))
        for t in range(M - 2, -1, -1):
            I[0][t] = psis[int(I[0][t + 1]) - 1][t + 1]
            if OUTPUT:
                print('i%d=psis%d(i%d)=%d' % (t + 1, t + 2, t + 2, I[0][t]))
        print("状态序列I：", I)
        return I.astype(int)

In [4]:
#习题10.1
I = [1, 2, 3]
O = ['红', '白']
A = [[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]]
B = [[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]]
# O = ['红', '白', '红', '红', '白', '红', '白', '白']
OL = ['红', '白', '红', '白']    #习题10.1的例子
PI = [[0.2, 0.4, 0.4]]

In [5]:
HMM = HiddenMarkov()
# HMM.forward(Q, V, A, B, O, PI)
# HMM.backward(Q, V, A, B, O, PI)
HMM.viterbi(I, O, A, B, OL, PI)
OL = ['红', '白', '白', '红', '红', '白', '红']    #例10.1的例子
HMM.viterbi(I, O, A, B, OL, PI)

状态序列I： [[3. 2. 2. 2.]]


array([[3, 2, 2, 2]])

状态序列I： [[3. 3. 3. 3. 3. 3. 3.]]


array([[3, 3, 3, 3, 3, 3, 3]])

In [6]:
I = [1, 2, 3]
O = ['红', '白']
A = [[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]]
B = [[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]]
OL = ['红', '白', '红', '红', '白', '红', '白', '白']
PI = [[0.2, 0.3, 0.5]]
# HMM.forward(I, O, A, B, OL, PI)
# HMM.backward(I, O, A, B, OL, PI)
HMM.viterbi(I, O, A, B, OL, PI)

状态序列I： [[3. 3. 3. 3. 2. 2. 2. 2.]]


array([[3, 3, 3, 3, 2, 2, 2, 2]])

In [7]:
import json
from operator import itemgetter
def load_dict(dict_path: str):
    # dict_path (str): 字典路径
    # Returns: dict: 字典
    with open(dict_path, "r", encoding="utf-8") as fr:
        return json.load(fr)

def load_data(data_path: str, char_dict: dict, label_dict: dict):
    # 加载数据
    # data_path (str): 数据路径
    # char_dict (str): 字符字典
    # label_dict (str): 标签字典
    # Returns:
    # seq_data: list，shape=(data_size,seq_len)
    # label_data: 形状和seq_data一样
    text_data = []
    label_data = []

    with open(data_path, 'r', encoding='utf-8') as fr:
        for idx, line in enumerate(fr.readlines()):
            line = eval(line)
            text = line['text']
            label = line['label']

            if len(text) > 1 and len(label) > 1 and len(text) == len(label):
                text = list(itemgetter(*text)(char_dict))
                label = list(itemgetter(*label)(label_dict))
                text_data.append(text)
                label_data.append(label)

    return text_data, label_data 

In [11]:
char_dict_path = 'G:/CDO/NLP/HMM_NER-master/data/char.dic'
label_dict_path = 'G:/CDO/NLP/HMM_NER-master/data/label.dic'
data_path = 'G:/CDO/NLP/HMM_NER-master/data/train_data.txt'

char_dict = load_dict(char_dict_path)
label_dict = load_dict(label_dict_path)
text_data, label_data = load_data(data_path, char_dict, label_dict)

In [12]:
i = 1
print('char_dict: ')
for k,v in char_dict.items():
    print('{}={}'.format(k,v),end=',')
    i += 1
    if i>10:
        break
i = 1
print('\nlabel_dict: ')
for k,v in label_dict.items():
    print('{}={}'.format(k,v),end=',')
    i += 1
    if i>10:
        break
type(text_data)
type(label_data)

char_dict: 
UNK=0,翠=1,左=2,绩=3,炫=4,锥=5,冢=6,毽=7,豢=8,返=9,
label_dict: 
B-ORG=0,B-PER=1,I-PER=2,O=3,I-LOC=4,B-LOC=5,I-ORG=6,

list

list

In [13]:
def estimate(char_dict,label_dict, text_data, label_data):
    # 训练模型
    # text_data (np.ndarray): 观测序列
    # label_data (np.ndarray): 隐藏状态
    # 估计转移概率矩阵

    I = [a for a in label_dict.values()]
    O = [a for a in char_dict.values()]
    n_chars = len(char_dict)
    n_labels = len(label_dict)

    # 初始化转移矩阵A、发射矩阵B、初始矩阵Pi
    transition = np.zeros((n_labels, n_labels))
    emission = np.zeros((n_labels, n_chars))
    pi = np.zeros(n_labels)

    # 偏置，用来防止log(0)或乘0的情况
    epsilon = 1e-8

    # 估计发射概率矩阵 初始概率矩阵

    print("estimate_transition_and_initial_probs")
    for line in label_data:
        # 统计初始状态
        pi[line[0]] += 1
        # 统计转移状态
        for cur, nxt in zip(line[:-1], line[1:]):
            transition[cur, nxt] += 1

    pi[pi == 0] = epsilon
    pi /= np.sum(pi)

    transition[transition == 0] = epsilon
    transition /= np.sum(transition, axis=1, keepdims=True)

    # 计算发射矩阵 P(Observation | Hidden_state)
    print("estimate_emission_probs")
    for seq_line, lbl_line in zip(text_data, label_data):
        for char, lbl in zip(seq_line, lbl_line):
            emission[lbl, char] += 1
    emission[emission == 0] = epsilon
    emission /= np.sum(emission, axis=1, keepdims=True)

    # 取log防止计算结果下溢
    #pi = np.log(pi)
    #transition = np.log(transition)
    #emission = np.log(emission)

    # Q, V, A, B, O, PI
    #I = list(range(n_labels))
    #O = list(range(n_chars))
    pi = [pi.tolist()]
    return I,O,transition, emission, pi 

In [14]:
I,O,A, B, PI = estimate(char_dict,label_dict, text_data, label_data)

estimate_transition_and_initial_probs
estimate_emission_probs


In [15]:
text = "主题之间割裂感太强，各期主题之间衔接感太差，最好是几次课可以围绕着一个主题来进行学习，从而在自主性学习和主题上都能够有很好的了解"

text_idx = itemgetter(*text)(char_dict)

text_idx = [int(a) for a in text_idx]

r = HMM.viterbi(I, O, A, B, text_idx, PI)
label_dict_r = {label_dict[k]:k for k in label_dict}
r = [label_dict_r[i-1] for i in r[0]]
for o,i in zip(text,r):
    print('{}\t{}'.format(o,i))

状态序列I： [[4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]]
主	O
题	O
之	O
间	O
割	O
裂	O
感	O
太	O
强	O
，	O
各	O
期	O
主	O
题	O
之	O
间	O
衔	O
接	O
感	O
太	O
差	O
，	O
最	O
好	O
是	O
几	O
次	O
课	O
可	O
以	O
围	O
绕	O
着	O
一	O
个	O
主	O
题	O
来	O
进	O
行	O
学	O
习	O
，	O
从	O
而	O
在	O
自	O
主	O
性	O
学	O
习	O
和	O
主	O
题	O
上	O
都	O
能	O
够	O
有	O
很	O
好	O
的	O
了	O
解	O


In [1]:
import numpy as np
from itertools import accumulate 

In [2]:
def _locate(prob_arr):
   # 给定概率向量，返回状态    seed = np.random.rand(1)
   for state, cdf in enumerate(accumulate(prob_arr)):
       if seed <= cdf:
           return state
   return 

In [15]:
class GenData:
   """    根据隐马尔科夫模型生成相应的观测数据  """
def __init__(self, hmm, n_sample):
        self.hmm = hmm
        self.n_sample = n_sample
def init_state(self):
       # 根据初始状态概率向量，生成初始状态        return _locate(self.hmm.S)
    def state_trans(self, current_state):
       # 转移状态        return _locate(self.hmm.A[current_state])
        def gen_obs(self, current_state):
       # 生成观测        return _locate(self.hmm.B[current_state])
            def gen_data(self):
       # 根据模型产生观测数据        current_state = self.init_state()
                start_obs = self.gen_obs(current_state)
                state = [current_state]
                obs = [start_obs]
                n = 0        
                while n < self.n_sample - 1:
                    n += 1            
                    current_state = self.state_trans(current_state)
                    state.append(current_state)
                    obs.append(self.gen_obs(current_state))
                return state, obs  

In [16]:
class HMM:
    def __init__(self, n_state, n_obs, S=None, A=None, B=None):
        self.n_state = n_state  
# 状态的个数n        self.n_obs = n_obs  # 观测的种类数m        self.S = S  # 1*n, 初始状态概率向量        self.A = A  # n*n, 状态转移概率矩阵        self.B = B  # n*m, 观测生成概率矩阵

In [18]:
def _alpha(hmm, obs, t):
   # 计算时刻t各个状态的前向概率    b = hmm.B[:, obs[0]]
    alpha = np.array([hmm.S * b])  # n*1    for i in range(1, t + 1):
    alpha = (alpha @ hmm.A) * np.array([hmm.B[:, obs[i]]])
    return alpha[0]

In [19]:
def forward_prob(hmm, obs):
   # 前向算法计算最终生成观测序列的概率, 即各个状态下概率之和    alpha = _alpha(hmm, obs, len(obs) - 1)
   return np.sum(alpha) 

In [20]:
def _beta(hmm, obs, t):
   # 计算时刻t各个状态的后向概率    beta = np.ones(hmm.n_state)
   for i in reversed(range(t + 1, len(obs))):
        beta = np.sum(hmm.A * hmm.B[:, obs[i]] * beta, axis=1)
   return beta 

In [21]:
def backward_prob(hmm, obs):
   # 后向算法计算生成观测序列的概率    beta = _beta(hmm, obs, 0)
   return np.sum(hmm.S * hmm.B[:, obs[0]] * beta) 

In [28]:
def fb_prob(hmm, obs, t=None):
   # 将前向和后向合并    if t is None:
    t = 0    
    res = _alpha(hmm, obs, t) * _beta(hmm, obs, t)
    return res.sum() 

In [29]:
def _gamma(hmm, obs, t):
   # 计算时刻t处于各个状态的概率    alpha = _alpha(hmm, obs, t)
    beta = _beta(hmm, obs, t)
    prob = alpha * beta
    return prob / prob.sum()

In [30]:
def point_prob(hmm, obs, t, i):
   # 计算时刻t处于状态i的概率    prob = _gamma(hmm, obs, t)
   return prob[i] 

In [31]:
def _xi(hmm, obs, t):
    alpha = np.mat(_alpha(hmm, obs, t))
    beta_p = _beta(hmm, obs, t + 1)
    obs_prob = hmm.B[:, obs[t + 1]]
    obs_beta = np.mat(obs_prob * beta_p)
    alpha_obs_beta = np.asarray(alpha.T * obs_beta)
    xi = alpha_obs_beta * hmm.A
    return xi / xi.sum() 

In [33]:
def fit(hmm, obs_data, maxstep=100):
   # 利用Baum-Welch算法学习    hmm.A = np.ones((hmm.n_state, hmm.n_state)) / hmm.n_state
    hmm.B = np.ones((hmm.n_state, hmm.n_obs)) / hmm.n_obs
    hmm.S = np.random.sample(hmm.n_state)  # 初始状态概率矩阵（向量），的初始化必须随机状态，否则容易陷入局部最优    hmm.S = hmm.S / hmm.S.sum()
    step = 0    
    while step < maxstep:
        xi = np.zeros_like(hmm.A)
        gamma = np.zeros_like(hmm.S)
        B = np.zeros_like(hmm.B)
        S = _gamma(hmm, obs_data, 0)
        for t in range(len(obs_data) - 1):
            tmp_gamma = _gamma(hmm, obs_data, t)
            gamma += tmp_gamma
            xi += _xi(hmm, obs_data, t)
            B[:, obs_data[t]] += tmp_gamma
       # 更新 A        for i in range(hmm.n_state):
            hmm.A[i] = xi[i] / gamma[i]
       # 更新 B        tmp_gamma_end = _gamma(hmm, obs_data, len(obs_data) - 1)
        gamma += tmp_gamma_end
        B[:, obs_data[-1]] += tmp_gamma_end
        for i in range(hmm.n_state):
            hmm.B[i] = B[i] / gamma[i]
       # 更新 S        hmm.S = S
        step += 1    
        return hmm  

In [34]:
def predict(hmm, obs):
   # 采用Viterbi算法预测状态序列    N = len(obs)
    nodes_graph = np.zeros((hmm.n_state, N), dtype=int)  
    # 存储时刻t且状态为i时， 前一个时刻t-1的状态，用于构建最终的状态序列    delta = hmm.S * hmm.B[:, obs[0]]  # 存储到t时刻，且此刻状态为i的最大概率    nodes_graph[:, 0] = range(hmm.n_state)
    for t in range(1, N):
        new_delta = []
        for i in range(hmm.n_state):
            temp = [hmm.A[j, i] * d for j, d in enumerate(delta)]  
            # 当前状态为i时， 选取最优的前一时刻状态            max_d = max(temp)
            new_delta.append(max_d * hmm.B[i, obs[t]])
            nodes_graph[i, t] = temp.index(max_d)
        delta = new_delta
    current_state = np.argmax(nodes_graph[:, -1])
    path = []
    t = N
    while t > 0:
        path.append(current_state)
        current_state = nodes_graph[current_state, t - 1]
        t -= 1     
        return list(reversed(path))

In [36]:
if __name__ == '__main__':
   # 预测    
    S = np.array([0.5, 0.5])
    A = np.array([[0.8, 1], [0.8, 0.8]])
    B = np.array([[0.2, 0.0, 0.8], [0, 0.8, 0.2]])
    hmm = HMM(2, 3, S, A, B)
    g = GenData(hmm, 200)
    state, obs = g.gen_data()
    print(obs)
    path = predict(hmm, obs)
    score = sum([int(i == j) for i, j in zip(state, path)])
    print(score / len(path))  

TypeError: GenData() takes no arguments

In [37]:
'''
https://github.com/Eilene/HMM-python/blob/master/HMM.py
实现隐马尔科夫模型的基本方法，
输入为状态转移矩阵，观测矩阵，初始状态概率向量，观测序列
实现前向算法和后向算法计算观测序列出现的概率
实现维特比算法找当前观测序列下最可能的状态序列
实现在给定模型和观测下，t时刻处于p状态的概率，t,p在main函数中指定
'''
from numpy import *

class HMM:

	def __init__(self):
		self.A=array([(0.1,0.4,0.5),(0.6,0.3,0.1)])
		self.B=array([(0.3,0.7),(0.6,0.4)])
		self.pi=array([(0.4),(0.6)])
		self.o=[0,1,0]
		self.t=len(self.o)#观测序列长度
		self.m=len(self.A)#状态集合个数
		self.n=len(self.B[0])#观测集合个数

	def qianxiang(self):
		#t时刻部分观测序列为o1,o2,ot,状态为qi的概率用矩阵x表示，
		#则矩阵大小行数为观测序列数，列数为状态个数
		self.x=array(zeros((self.t,self.m)))
		#先计算出时刻1时，观测为o1,状态为qi的概率
		for i in range(self.m):
			self.x[0][i]=self.pi[i]*self.B[i][self.o[0]]
		for j in range(1,self.t):
			for i in range(self.m):
				#前一时刻所有状态的概率乘以转移概率得到i状态概率
				#i状态的概率*i状态到j观测的概率
				temp=0
				for k in range(self.m):
					temp=temp+self.x[j-1][k]*self.A[k][i]
				self.x[j][i]=temp*self.B[i][self.o[j]]
		result=0
		for i in range(self.m):
			result=result+self.x[self.t-1][i]
		print("前向概率矩阵及当前观测序列概率如下：")
		print(self.x)
		print(result)

	def houxiang(self):
		#t时刻状态为qi,从t+1到T观测为ot+1,ot+2,oT的概率用矩阵y表示
		#则矩阵大小行数为观测序列数，列数为状态个数
		self.y=array(zeros((self.t,self.m)))
		#下面为对最终时刻的所有状态，接下来的观测序列概率初始化为1
		#(可以理解为接下来没有观测所有为1)
		for i in range(self.m):
			self.y[self.t-1][i]=1
		j=self.t-2
		#j时刻为i状态，转移到k状态，k状态观测为oj+1,
		#再乘以j+1时刻状态为k的B矩阵的值，对k遍历相加，
		#即为j时刻i状态前提下，后面满足观测序列的概率
		while(j>=0):
			for i in range(self.m):
				for k in range(self.m):
					self.y[j][i]+=self.A[i][k]*self.B[k][self.o[j+1]]*self.y[j+1][k]
			j=j-1
		#第一个状态任意，观测为o1,所以对所有第一个状态概率相加
		result=0
		for i in range(self.m):
			result=result+self.pi[i]*self.B[i][self.o[0]]*self.y[0][i]
		print('后向概率矩阵及当前观测序列概率如下：')
		print(self.y)
		print(result)

	def get_stateprobability(self,t,p):
		#打印在观测为self.o的前提下，t时刻，处于状态p的概率,
		#self.x[t][p]表示到t时刻观测为o1,o2,ot,状态为p的概率
		#self.y[t][p]表示在t时刻状态为p的前提下，接下来观测为ot+1,ot+2,oT的概率
		#self.x[t][p]*self.y[t][p]即表示观测为self.o，且t时刻处于状态p的概率,
		#利用贝叶斯公式，除以观测为self.o的概率即为所求
		if(t>self.t or p>self.m):
			print(u'输入数据超过范围')
			return
		print('在时刻'+str(t)+u'处于状态'+str(p)+u'的概率是：')
		temp=self.x[t-1][p-1]*self.y[t-1][p-1]
		total=0
		for i in range(self.m):
			total=total+self.x[t-1][i]*self.y[t-1][i]
		print(temp/total)

	def viterbi(self):
		#利用模型和观测序列找出最优的状态序列
		#时刻t时，很多路径可以到达状态i,且观测为self.o,
		#每个路径都有自己的概率，最大的概率用矩阵z记录,前一个状态用d矩阵记录
		self.z=array(zeros((self.t,self.m)))
		self.d=array(zeros((self.t,self.m)))
		for i in range(self.m):
			self.z[0][i]=self.pi[i]*self.B[i][self.o[0]]
			self.d[0][i]=0
		for j in range(1,self.t):
			for i in range(self.m):
				maxnum=self.z[j-1][0]*self.A[0][i]
				node=1
				for k in range(1,self.m):
					temp=self.z[j-1][k]*self.A[k][i]
					if(maxnum<temp):
						maxnum=temp
						node=k+1
				self.z[j][i]=maxnum*self.B[i][self.o[j]]
				self.d[j][i]=node
		#找到T时刻概率最大的路径
		max_probability=self.z[self.t-1][0]
		last_node=[1]
		temp=0
		for i in range(1,self.m):
			if(max_probability<self.z[self.t-1][i]):
				max_probability=self.z[self.t-1][i]
				last_node[0]=i+1
				temp=i
		i=self.t-1
		#self.d[t][p],t时刻状态为p的时候，t-1时刻的状态
		while(i>=1):
			last_node.append(self.d[i][temp])
			i=i-1
		temp=['o']
		temp[0]=int(last_node[len(last_node)-1])
		j=len(last_node)-2
		while j>=0:
			temp.append(int(last_node[j]))
			j=j-1
		print('路径节点分别为')
		print(temp)
		print('该路径概率为'+str(max_probability))

In [38]:
if __name__ == '__main__':
	test=HMM()
	test.qianxiang()
	test.houxiang()
	test.get_stateprobability(3,3)
	test.viterbi() 

前向概率矩阵及当前观测序列概率如下：
[[0.12     0.36    ]
 [0.1596   0.0624  ]
 [0.01602  0.049536]]
0.065556
后向概率矩阵及当前观测序列概率如下：
[[0.0765 0.1566]
 [0.27   0.36  ]
 [1.     1.    ]]
0.065556
输入数据超过范围
路径节点分别为
[2, 1, 2]
该路径概率为0.036288
